In [1]:
import torch
import numpy as np
import pandas as pd
from tqdm import tqdm
import json
from torch.utils.data import TensorDataset, DataLoader
import torch.optim as optim
import math

In [2]:
# getting the prepared graph from "preparingGraphs"
df = pd.read_csv('linkedGraphsWithSolutions.csv', engine='python')

In [3]:
size_of_matrixs = int(math.sqrt(len(json.loads(df.iloc[0]["Matrix"]))))
amount_of_matrixs = len(df)

In [4]:
# creating a nn model
model = torch.nn.Sequential(
    torch.nn.Linear(size_of_matrixs*size_of_matrixs, 150),
    torch.nn.ReLU(),
    torch.nn.Linear(150, 150),
    torch.nn.ReLU(),
    torch.nn.Linear(150, 1),
)

In [5]:
matrix_table = []
amount_of_sol = []
# puting the columns in an array
for i in range(len(df)):
    current = df.iloc[i]
    matrix_table.append(json.loads(current["Matrix"]))
    amount_of_sol.append([current["Amount"]])

In [6]:
# spliting the data to train and test
test_size = int(0.3 * amount_of_matrixs)
x_test = torch.tensor(matrix_table[:test_size]).type(torch.FloatTensor)
y_test = torch.tensor(amount_of_sol[:test_size]).type(torch.FloatTensor)
x_train = torch.tensor(matrix_table[test_size:]).type(torch.FloatTensor)
y_train = torch.tensor(amount_of_sol[test_size:]).type(torch.FloatTensor)

In [7]:
# putting the data in data loaders
my_train_dataset = TensorDataset(x_train, y_train)
train_data = DataLoader(my_train_dataset, shuffle = True)
my_test_dataset = TensorDataset(x_test, y_test)
test_data = DataLoader(my_test_dataset, shuffle = True)

In [8]:
optimizer = optim.Adam(model.parameters(), lr=0.01)
criterion = torch.nn.MSELoss()
'''
trains the model
'''
def train():
    model.train()
    for idx, (data, labels) in tqdm(enumerate(train_data)):
        optimizer.zero_grad()
        out = model(data)
        loss = criterion(out, labels)
        loss.backward()
        optimizer.step()

In [9]:
'''
tests the model
'''
def test():
    model.eval()
    valid_loss = 0
    for idx, (data, labels) in tqdm(enumerate(test_data)):
        out = model(data)
        # sum up batch loss
        valid_loss += criterion(out, labels).item()
    print("Result: ", valid_loss / len(test_data))

In [10]:
epoch = 3
for i in range(epoch):
    train()
    test()

14000it [00:35, 391.12it/s]
6000it [00:01, 4250.87it/s]
35it [00:00, 347.46it/s]

Result:  0.6717143412714387


14000it [00:36, 382.32it/s]
6000it [00:01, 4375.04it/s]
88it [00:00, 423.92it/s]

Result:  0.5356983100723952


14000it [00:33, 417.28it/s]
6000it [00:01, 5087.17it/s]

Result:  0.5343206727575042
